# FIFA 23 - Ultimate team players

Accessing the Futbin website to collect data of Players in FIFA 23 Ultimate Team.\

### Libs
- Pandas - To load and manipulate data
- Scrapy - To scrap data from a website

In [1]:
import pandas as pd
from selenium import webdriver    
from datetime import  date
from scrapy import Selector
import time
import warnings
warnings.simplefilter(action='ignore', category=DeprecationWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)
import random

# defining today to use in some cases
today = date.today()

df_players = pd.DataFrame()

# Step 1 - Iterate over pages to get players data and sabe it on dataframe
We need to instanciate a new driver every iteration to by-pass some anti-scrap blockers.

In [4]:
# variable to pagination
page_number = 1

for page in range(2):
    # Creating a webdriver instance
    driver = webdriver.Chrome(r"Users\Felipe Mayer\Downloads\chromedriver_win32")
    url = 'https://www.futbin.com/players?page=' + str(page_number)
            
    driver.get(url)
    time.sleep(random.randint(2, 5))

    src = driver.page_source
    sel = Selector(text=src)

    players = sel.xpath('.//thead/following-sibling::tbody/tr[starts-with(@class, "player_tr")]')

    for player in players:
        index = len(df_players.index)
        df_players.loc[index, 'Name'] = player.xpath('.//a[contains(@class, "player_name_players_table")]/text()').get(default='')
        df_players.loc[index, 'Team'] = player.xpath('.//span[@class="players_club_nation"]/a[1]/@data-original-title').get(default='')
        df_players.loc[index, 'Country'] = player.xpath('.//span[@class="players_club_nation"]/a[2]/@data-original-title').get(default='')
        df_players.loc[index, 'League'] = player.xpath('.//span[@class="players_club_nation"]/a[3]/@data-original-title').get(default='')
        df_players.loc[index, 'Rating'] = player.xpath('.//span[contains(@class, "rating")]/text()').get(default='')
        rating_type = player.xpath('.//span[contains(@class, "rating")]/@class').get(default='')
        s = 'ut23'
        ss = rating_type.split(s, 1)
        df_players.loc[index, 'Rare'] = rating_type.split(s, 1)[1].split()[-1]
        df_players.loc[index, 'Version_color'] = ss[1].split(df_players.loc[index, 'Rare'], 1)[0]
        df_players.loc[index, 'Position'] = player.xpath('./td[4]/div[1]/text()').get(default='')
        df_players.loc[index, 'Other_Positions'] = player.xpath('./td[4]/div[2]/text()').get(default='')
        df_players.loc[index, 'Version'] = player.xpath('./td[5]/div[1]/text()').get(default='')
        df_players.loc[index, 'Run_type'] = player.xpath('./td[5]/div[2]/a/text()').get(default='').replace('/n', ' ').strip()
        df_players.loc[index, 'Price'] = player.xpath('./td[6]/span/text()').get(default='')
        df_players.loc[index, 'Skill'] = player.xpath('./td[7]/text()').get(default='')
        df_players.loc[index, 'Weak_foot'] = player.xpath('./td[8]/text()').get(default='')
        df_players.loc[index, 'Attack_rate'] = player.xpath('./td[9]/span[1]/text()').get(default='')
        df_players.loc[index, 'Defense_rate'] = player.xpath('./td[9]/span[2]/text()').get(default='')
        df_players.loc[index, 'Pace'] = player.xpath('./td[10]/span/text()').get(default='')
        df_players.loc[index, 'Shoot'] = player.xpath('./td[11]/span/text()').get(default='')
        df_players.loc[index, 'Pass'] = player.xpath('./td[12]/span/text()').get(default='')
        df_players.loc[index, 'Drible'] = player.xpath('./td[13]/span/text()').get(default='')
        df_players.loc[index, 'Defense'] = player.xpath('./td[14]/span/text()').get(default='')
        df_players.loc[index, 'Physical'] = player.xpath('./td[15]/span/text()').get(default='')
        df_players.loc[index, 'Body_type'] = player.xpath('./td[16]/div[2]/a/text()').get(default='')
        df_players.loc[index, 'Height_cm'] = player.xpath('./td[16]/div[1]/text()').get(default='').split('|')[0].replace('cm', ' ').strip()
        weight = player.xpath('string(./td[16]/div[2])').get(default='')
        if weight != '':
            df_players.loc[index, 'Weight (kg)'] = weight.split('(')[1].replace('kg)', ' ').strip()
        df_players.loc[index, 'Popularity'] = player.xpath('./td[17]/text()').get(default='')
        df_players.loc[index, 'Base_Stats'] = player.xpath('./td[18]/text()').get(default='')
        df_players.loc[index, 'In_Game_Stats'] = player.xpath('./td[19]/text()').get(default='')
        df_players.loc[index, 'Game_version'] = 'FIFA 23'
        df_players.loc[index, 'Updated_at'] = today

    driver.close()

    page_number += 1

display(df_players)

,Name,Team,Country,League,Rating,Rare,Version_color,Position,Other_Positions,Version,...,Defense,Physical,Body_type,Height_cm,Weight (kg),Popularity,Base_Stats,In_Game_Stats,Game_version,Updated_at
0,Achraf Hakimi,Paris SG,Morocco,Ligue 1,94,rare,toty gold,RB,RWB,TOTY,...,90,91,Lean,181,73,1654,542,2580,FIFA 23,2023-03-10
1,Éder Militão,Real Madrid,Brazil,LaLiga Santander,94,rare,toty gold,CB,RB,TOTY,...,95,92,Lean,186,78,1047,501,2362,FIFA 23,2023-03-10
2,Theo Hernández,Milan,France,Serie A TIM,94,rare,toty gold,LB,LWB,TOTY,...,88,92,Unique,184,81,2574,536,2514,FIFA 23,2023-03-10
3,David Beckham,FUT ICONS,England,Icons,94,rare,toty_icon gold,RM,"CM,RW",TOTY Icon,...,70,83,Average,182,74,1394,519,2528,FIFA 23,2023-03-10
4,Ruud Gullit,FUT ICONS,Netherlands,Icons,94,rare,toty_icon gold,CF,"CM,CAM,ST",TOTY Icon,...,84,91,Unique,191,88,1374,538,2613,FIFA 23,2023-03-10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,Hugo Sánchez,FUT ICONS,Mexico,Icons,93,rare,toty_icon gold,ST,CF,TOTY Icon SBC,...,45,78,Average,176,73,5919,486,2363,FIFA 23,2023-03-10
86,Gareth Bale,LAFC,Wales,Major League Soccer,93,rare,sbc_premium gold,RW,"LB,LWB,RM",Premium SBC,...,72,86,Unique,185,82,13407,524,2555,FIFA 23,2023-03-10
87,Sadio Mané,FC Bayern,Senegal,Bundesliga,93,rare,sbc_flashback gold,LW,"LM,CF",Flashback SBC,...,49,83,Unique,174,69,3706,492,2366,FIFA 23,2023-03-10
88,Kevin De Bruyne,Manchester City,Belgium,Premier League,93,rare,if gold,CM,CAM,SIF,...,68,80,Unique,181,70,454,499,2407,FIFA 23,2023-03-10


# Step 2 - Save it on a CSV to be used in futher analysis

In [20]:
df_players.to_csv('players.csv', index=False)